In [1]:
import emcee
import corner
from multiprocessing import Pool, cpu_count
print('There are: '+str(cpu_count())+' CPUs')

There are: 8 CPUs


In [ ]:

##########################################
##### Define Prior and Loglikelihood #####
##########################################

# theta:
# - kp
# - vsys
# - a

N = wave.shape[1]
f = flux - np.nanmean(flux, axis=1)[:, np.newaxis]
f2 = np.nansum(f**2, axis=1)

phase = inset phase here
pscale = np.sin(2*np.pi*phase)

def prior(theta):
    
    # stop velocity wandering off
    if theta[0]>400 or theta[0]<-400: return False
    if theta[1]>400 or theta[1]<-400: return False
    
    return True

def log_prob(theta):

    if prior(theta)==False: return - np.inf

    mod = np.zeros(flux.shape)
    planspec = # planet spec function
    for k in range(mod.shape[0]):
        mod[k] = theta[2]*planspec(wave[k]*( 1 - (theta[1] + theta[0]*pscale[k])/(3*10**5)))
    
    g = mod - np.nanmean(mod, axis=1)[:, np.newaxis]
    #g /= np.nanstd(mod, axis=1)[:, np.newaxis]
    
    fg = np.nansum(f*g, axis=1)
    g2 = np.nansum(g**2, axis=1)
    logl = - N/2 * np.nansum( np.log(1/N * (f2 - 2*fg + g2)) )

    return logl
    


In [ ]:
#####################################
##### Initial params and labels #####
#####################################

med = [150, 0, 1.0]
std = [10, 10, 0.05]

labels = ['Kp', 'Vsys', 'a']


In [ ]:
###############################
##### Run MCMC (too slow) #####
###############################

ndim, nwalkers = 3, 6
p0 = np.random.randn(nwalkers, ndim)
for i in range(ndim):
    p0[:,i] = p0[:,i]*std[i] + med[i]
    
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, pool=pool)
    sampler.run_mcmc(p0, 200, progress=True)
    


In [ ]:
#############################
##### Check MCMC chains #####
#############################

fig, axes = plt.subplots(ndim, figsize=(10, 10), sharex=True)
samples = sampler.get_chain()
cut=0 #remove initial n steps from plot

for i in range(ndim):
    ax = axes[i]
    ax.plot(np.arange(len(samples))[cut:], samples[cut:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])

axes[-1].set_xlabel("step number")
plt.savefig('./chains.png')
plt.show()
plt.close()


In [ ]:
###############################
##### Plot output of MCMC #####
###############################

n = 40 #burn in
flat_samples = sampler.get_chain(discard=n, flat=True)
figure = corner.corner(flat_samples, labels=labels, show_titles=True, title_kwargs={"fontsize": 10})
plt.savefig('./corner.png')
plt.show()
plt.close()

hd = ''
for l in labels: hd += l +','
np.savetxt('./chains.csv', flat_samples, delimiter=",", header=hd[:-1])